Họ và tên: **Nguyễn Ngọc Băng Tâm**

MSSV: **1712747**

In [2]:
import os

from requests_html import HTMLSession
import requests
import pandas as pd
import re
import time

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

import urllib.robotparser # Kiểm tra file robot.txt có được phép crawl không

In [3]:
if not os.path.exists('Api_data/'):
    os.makedirs('Api_data/')
    
if not os.path.exists('Crawl_data/'):
    os.makedirs('Crawl_data/')
    
if not os.path.exists('Crawl_urls/'):
    os.makedirs('Crawl_urls/')

## 2. Parse HTML

**Ý tưởng chọn tập mẫu:**

- B1: Tìm và lưu lại **top 500 từ khóa được tìm kiếm nhiều nhất (popular searches) trong 24 giờ gần nhất** vào file `popular_keywords.txt`. 
- B2: Lọc lại 300 từ khóa và lấy các url danh sách các playlist thuộc từ khóa đó, lưu vào file `query_popular_playlists.txt`. 
- B3: Parse playlist url từ các url kết quả truy vấn và lưu vào file `popular_playlists.txt`
- B4: Với mỗi playlist url, parse user url và lưu vào file `popular_users.txt`
- B5: Với mỗi playlist url, parse 5 track url thuộc playlist đó và lưu vào file `popular_tracks.txt`

**Lưu ý:**
- Ta sẽ xử lý các dòng trùng lắp bằng cách thêm các url vào một tập hợp (set) và lưu file từ các phần tử của tập hợp đó
- Lưu tất cả các url trên vào các file text trong thư mục trung gian `Crawl_urls`, khi cần thực hiện parse HTML chỉ cần duyệt lấy url và parse thành từng đối tượng user, track và playlist

### a. Tạo robot parser

In [3]:
rp = urllib.robotparser.RobotFileParser()
rp.set_url('https://soundcloud.com/robots.txt')
rp.read()

### b. Lấy các url cần thiết trước khi parse

#### B1. Lấy 500 từ khóa được tìm kiếm nhiều nhất trong 24 giờ qua
Thời điểm đồ án chọn là 24 giờ qua tính đến ngày 05/11/2020

In [4]:
def get_popular_keywords(popular_keywords_file, sleep_time=1):
    """ 
    Get the latest 500 hot keywords
    """
    session = HTMLSession()
    
    with open(popular_keywords_file, 'w') as out:
        for page in range(1, 6):
            # sleep before request a new session
            time.sleep(sleep_time)
            base_url = f'https://soundcloud.com/popular/searches/{page}'
            r = session.get(base_url)
            for url in r.html.absolute_links:
                if "soundcloud.com/search" in url and rp.can_fetch('*', url):
                    out.write(f'{url}\n')

In [5]:
get_popular_keywords('Crawl_urls/popular_keywords.txt')

#### B2. Lấy 300 url chứa kết quả truy vấn các playlist phổ biến

In [1]:
def get_popular_query_results(popular_keywords_file, data_type):
    """ 
    Get query results of playlists from the latest 300 keyword (based on the data_type argumewith open(f'{data_type}_url.txt', 'w') as out:nt)
    - popular_keywords_file: txt file of the urls of 300 popular keywords
    - data_type: type of objects to get url
    """
    
    # dict - equivalent Soundcloud terminology of each data_type
    soundcloud_term = {'users': 'people',
                   'tracks': 'sounds',
                   'playlists': 'sets'}
    
    with open(f'Crawl_urls/query_popular_{data_type}.txt', 'w') as out:
        with open(popular_keywords_file, 'r') as inp:
            cnt = 0
            for line in inp:

                if cnt == 300:
                    break
        
                if rp.can_fetch('*', line):
                    preprocessed_line = line.replace("?q", f'/{soundcloud_term[data_type]}?q')
                    out.write(f'{preprocessed_line}')
                    cnt += 1

In [60]:
get_popular_query_results('Crawl_urls/popular_keywords.txt', 'playlists')

#### B3. Lấy url của từng playlist dựa trên url kết quả truy vấn

In [10]:
def get_data_type_urls(data_type, data_regex, sleep_time=1):
    """
    Get urls of each user, playlist, track
    - data_type: user / playlist / track
    - data_regex: regular expression to match the data_type url
    """
    session = HTMLSession()

    with open(f'Crawl_urls/popular_{data_type}.txt', 'w') as out:
        with open(f'Crawl_urls/query_popular_{data_type}.txt', 'r') as inp:
            for line in inp:
                if rp.can_fetch('*', line):
                     # sleep before request a new session
                    time.sleep(sleep_time)
                    r = session.get(line)
                     
                    for url in r.html.absolute_links:                
                        if 'search' not in url and re.search(data_regex, url) is not None:
                            out.write(f'{url}\n')  

In [11]:
playlist_regex = 'https?:\/\/soundcloud.com\/\S+'

In [13]:
# Playlist
get_data_type_urls('playlists', playlist_regex)

Xóa các dòng playlist trùng lắp và kiểm tra số lượng playlist duy nhất

In [64]:
# TEST PLAYLIST
unique_playlists = set()

# Thực hiện xóa các dòng trùng lắp
def remove_duplicates():
    with open('Crawl_urls/unique_popular_playlists.txt', 'w') as out:
        with open('Crawl_urls/popular_playlists.txt', 'r') as inp:
            for line in inp:
                unique_playlists.add(line)
        for playlist in unique_playlists:
            out.write(f'{playlist}')
            
remove_duplicates()

In [65]:
with open('Crawl_urls/unique_popular_playlists.txt', 'r') as inp:
    cnt = 0
    for line in inp:
        cnt += 1
        
print(f'unique playlists: {cnt}')

unique playlists: 2722


#### B4: Với mỗi playlist url, parse user url và lưu vào file `popular_users.txt`

In [5]:
def get_users_from_playlist_urls():
    
    # delete duplicate lines in user_file
    unique_users = set()
    with open('Crawl_urls/unique_popular_users.txt', 'w') as out:
        with open('Crawl_urls/unique_popular_playlists.txt', 'r') as inp:
            for line in inp:
                if rp.can_fetch('*', line):
                    line_lst = line.split('/')[:4]
                    user_url = '/'.join(line_lst)
                    unique_users.add(user_url)
                    
        for user in unique_users:
            out.write(f'{user}\n')

In [ ]:
get_users_from_playlist_urls()

#### B5: Với mỗi playlist url, parse 5 track url thuộc playlist đó và lưu vào file `popular_tracks.txt`

Do các url của Soundcloud có nhúng Javascript, ta thực hiện download mã nguồn trang web bằng selenium và parse các đối tượng cần lấy bằng regex

In [4]:
def get_tracks_from_playlist_urls(driver, sleep_time=1):

    unique_tracks = set()
    
    with open('Crawl_urls/unique_popular_tracks.txt', 'w') as out:
        with open('Crawl_urls/unique_popular_playlists.txt', 'r') as inp:
            for line in inp:
                if rp.can_fetch('*', line):
                    # sleep before requesting a new session
                    time.sleep(sleep_time)
                    driver.get(line)
                    page_src = browser.page_source

                    # find playlist
                    playlist_regex = '\[{"artwork_url".+"}\]'
                    pos = re.search(playlist_regex, page_src)
                
                    if pos is not None:
                        raw_str = page_src[pos.start():pos.end()]
                
                        # find track url
                        track_search = r'"permalink_url":"https:\S+?"{1}'
                        tracks = re.findall(track_search, raw_str)

                        if len(unique_tracks) == 6000:
                            break
                    
                        for track in tracks:
                            if track.count('/') == 4:
                                track = track[len('"permalink_url":"'):-1]
                                unique_tracks.add(track)
                    else:
                        print(f'{line} has problems')
                else:
                    print(f"Can't fetch {line}")
                
        
        for track in unique_tracks:
            out.write(f'{track}\n')

In [15]:
chrome_options = Options()
chrome_options.add_argument("--headless")

browser = webdriver.Chrome('./chromedriver', options=chrome_options)

get_tracks_from_playlist_urls(browser)

https://soundcloud.com/18-user-user/sets/die-very-rough
 has problems
https://soundcloud.com/18-user-user/sets/big-bootie-mix-bottie
 has problems
https://soundcloud.com/2020-songs/sets/what-do-you-know-about-love-pop-smoke
 has problems
https://soundcloud.com/dll-repost-2/sets/what-do-you-know-about-love-pop-smoke-2
 has problems
https://soundcloud.com/2020-songs/sets/whoopty-cj
 has problems
https://soundcloud.com/18-user-user/sets/big-bootie-mix
 has problems
https://soundcloud.com/2020-songs/sets/easy-life-sangria-ft-arlo-parks
 has problems
https://soundcloud.com/18-user-user/sets/drip-like-me
 has problems
https://soundcloud.com/philipjcastillo/sets/dolly-lil-uzi-vert
 has problems
https://soundcloud.com/2020-songs/sets/tik-tok
 has problems
https://soundcloud.com/319963465-79694589-weeee/sets/trippie-redd
 has problems
https://soundcloud.com/richardmloleary/sets/capitalize-off-pain
 has problems
https://soundcloud.com/user-1920291817281910/sets/laugh-now-cry-later
 has problems


## Tham khảo 
- https://www.oreilly.com/library/view/web-scraping-with/9781491910283/ch01.html
- https://towardsdatascience.com/a-practical-guide-to-exploratory-data-analysis-spotify-dataset-d8f703da663e